In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from transformers.image_utils import load_image
from datasets import features, load_dataset

### Use the checkpoint path of desired model

In [ ]:
model_id = "checkpoints/SFT_256M_smolcot_mini_3k"

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "CPU"
DEVICE

from accelerate import PartialState
device_string = PartialState().process_index



processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, device_map={'':device_string}
)

DEVICE = model.device

### Either use any data or any individual example

In [ ]:
# data = Dataset.load_from_disk("../data/eval_smolcot_hf")
# len(data)


qs = "How many people are in the image?"
image = None



In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": f"{qs}"}
        ]
    },
]

prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
with torch.no_grad():
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    inputs = inputs.to(DEVICE)
    
    generated_ids = model.generate(**inputs
    # generated_ids = model.generate(**inputs
                                   , max_new_tokens = 256
                                   )
    generated_texts = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
    
    )

print("----------------\nOutput:\n----------------")
print(generated_texts[0])